# Aula05.Ex03 - Exemplo - Esfera - Transformação Geométrica 3D

### Primeiro, vamos importar as bibliotecas necessárias.

In [3]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
import random

### Inicializando janela

In [5]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

### Shaders

Note que agora usamos vec3, já que estamos em 3D.

In [8]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

In [9]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [11]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [13]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [15]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [17]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [19]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [21]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


#### Modelando uma esfera

In [24]:
PI = 3.141592
r = 0.5 # raio
num_sectors = 32 # qtd de sectors (longitude)
num_stacks = 32 # qtd de stacks (latitude)


# grid sectos vs stacks (longitude vs latitude)
sector_step=(PI*2)/num_sectors # variar de 0 até 2π
stack_step=(PI)/num_stacks # variar de 0 até π

# Entrada: angulo de longitude, latitude, raio
# Saida: coordenadas na esfera
def F(u,v,r):
    x = r*math.sin(v)*math.cos(u)
    y = r*math.sin(v)*math.sin(u)
    z = r*math.cos(v)
    return (x,y,z)

# vamos gerar um conjunto de vertices representantes poligonos
# para a superficie da esfera.
# cada poligono eh representado por dois triangulos
vertices_list = []
for i in range(0,num_sectors): # para cada sector (longitude)
    for j in range(0,num_stacks): # para cada stack (latitude)
        
        
        
        u = i * sector_step # angulo setor
        v = j * stack_step # angulo stack
        
        un = 0 # angulo do proximo sector
        if i+1==num_sectors:
            un = PI*2
        else: un = (i+1)*sector_step
            
        vn = 0 # angulo do proximo stack
        if j+1==num_stacks:
            vn = PI
        else: vn = (j+1)*stack_step
        
        # vertices do poligono
        p0=F(u, v, r)
        p1=F(u, vn, r)
        p2=F(un, v, r)
        p3=F(un, vn, r)
        
        # triangulo 1 (primeira parte do poligono)
        vertices_list.append(p0)
        vertices_list.append(p2)
        vertices_list.append(p1)
        
        # triangulo 2 (segunda e ultima parte do poligono)
        vertices_list.append(p3)
        vertices_list.append(p1)
        vertices_list.append(p2)


total_vertices = len(vertices_list)
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)

In [25]:
for v in vertices:
    print(v)

([0. , 0. , 0.5],)
([0. , 0. , 0.5],)
([0.04900856, 0.        , 0.49759236],)
([0.04806687, 0.00956109, 0.49759236],)
([0.04900856, 0.        , 0.49759236],)
([0. , 0. , 0.5],)
([0.04900856, 0.        , 0.49759236],)
([0.04806687, 0.00956109, 0.49759236],)
([0.09754514, 0.        , 0.49039266],)
([0.09567084, 0.01903011, 0.49039266],)
([0.09754514, 0.        , 0.49039266],)
([0.04806687, 0.00956109, 0.49759236],)
([0.09754514, 0.        , 0.49039266],)
([0.09567084, 0.01903011, 0.49039266],)
([0.1451423 , 0.        , 0.47847018],)
([0.14235345, 0.02831585, 0.47847018],)
([0.1451423 , 0.        , 0.47847018],)
([0.09567084, 0.01903011, 0.49039266],)
([0.1451423 , 0.        , 0.47847018],)
([0.14235345, 0.02831585, 0.47847018],)
([0.19134168, 0.        , 0.46193978],)
([0.1876651 , 0.0373289 , 0.46193978],)
([0.19134168, 0.        , 0.46193978],)
([0.14235345, 0.02831585, 0.47847018],)
([0.19134168, 0.        , 0.46193978],)
([0.1876651 , 0.0373289 , 0.46193978],)
([0.23569833, 0.       

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot (buffer).

In [27]:
# Request a buffer slot from GPU
buffer_VBO = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [29]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)

### Associando variáveis do programa GLSL (Vertex Shader) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [31]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [33]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui <b> três </b> coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [35]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

### Vamos pegar a localização da variável color para que possamos definir a cor em nosso laço da janela!

In [37]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, nós exibimos a janela!


In [39]:
glfw.show_window(window)

### Loop principal da janela.

In [41]:
anguloRotacao = 0.0

glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c


while not glfw.window_should_close(window):
    
    anguloRotacao -= 0.01 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(anguloRotacao)
    sin_d = math.sin(anguloRotacao)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_rotation_z = np.array([     cos_d, -sin_d, 0.0, 0.0, 
                                    sin_d,  cos_d, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_d, -sin_d, 0.0, 
                                    0.0, sin_d,  cos_d, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_d,  0.0, sin_d, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_d, 0.0, cos_d, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_x)
    mat_transform = multiplica_matriz(mat_rotation_y,mat_transform)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    
    for triangle in range(0,len(vertices),3):
       
        random.seed( triangle )
        R = random.random()
        G = random.random()
        B = random.random()  
        glUniform4f(loc_color, R, G, B, 1.0)
        
        glDrawArrays(GL_TRIANGLES, triangle, 3)     
    
    glfw.swap_buffers(window)
    glfw.poll_events()

glfw.terminate()

# Exercício

Modifique esse código para aplicar outras transformações (de sua escolha) conforme eventos do teclado.